##Lecture des données

In [0]:
import pandas as pd

f_eveler = "/content/drive/My Drive/EEW/Data/eveler/narbonne_mars2020.pkl"
f_lerne  = "/content/drive/My Drive/EEW/Data/lerne/narbonne_2020-03.csv"




df_eveler = pd.read_pickle(f_eveler)

df_lerne = pd.read_csv(f_lerne,sep = ",",low_memory = False)
df_lerne.drop_duplicates(inplace=True)
df_lerne.sort_values(by=['mesure_date_val'],inplace=True)
df_lerne.index = range(len(df_lerne))
df_lerne = df_lerne.pivot_table(index='mesure_date_val',columns='equipement_nom',values='mesure_value')
df_lerne.index = pd.to_datetime(df_lerne.index,format="%Y-%m-%d %H:%M:%S %Z")




# Consommation journalière en kWh/jour Eveler

#df_eveler_conso_jour = df_eveler['P_ACT_KW'].resample('D').mean()*24
# somme des consommations du sous-comptage



## Temps de marche au pas de temps horaire

In [0]:
import sys
sys.path.append("/content/drive/My Drive/EEW")
import imp
import Function_sous_comptage_func 
imp.reload(Function_sous_comptage_func)

from Function_sous_comptage_func import Compute_Equi_TDM_Lerne
Periode = '1H' # Fréquence d'aggrégation des résultats
label_MA = [l for l in df_lerne.columns if l.startswith("M_")]
# Temps de fonctionnement en heure
Equip_Work_Time = Compute_Equi_TDM_Lerne(df_lerne, label_MA, periode = Periode)


## Fusion données Eveler et TDM

In [0]:
# Consommation horaire en kWh Eveler

df_eveler_conso_heure = df_eveler.resample('H').mean()
df_eveler_conso_heure.drop([df_eveler_conso_heure.index[0],df_eveler_conso_heure.index[-1]],inplace=True)


data = df_eveler_conso_heure.merge(Equip_Work_Time, how='outer', left_index = True, right_index = True)

data.drop(columns="P_REACT_KVAR_POSITIVE", inplace=True)
data.dropna(inplace=True)

In [0]:
X_TDM = data.drop(columns='P_ACT_KW').values
Y_CONSO = data['P_ACT_KW'].values

## Ridge regression with feature normalization

In [36]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X_TDM, Y_CONSO,
                                                   random_state = 0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for alpha in [0,0.001, 0.1, 0.5,1,2, 3, 5, 10, 20, 50]:

  linridge = Ridge(alpha=alpha).fit(X_train_scaled, y_train)


#print('ridge regression linear model intercept: {}'
#     .format(linridge.intercept_))
#print('ridge regression linear model coeff:\n{}'
#     .format(linridge.coef_))
  print('R-squared score (training): {:.3f}'
      .format(linridge.score(X_train_scaled, y_train)))
  print('R-squared score (test): {:.3f}'
      .format(linridge.score(X_test_scaled, y_test)))
  print('Number of non-zero features: {}'
      .format(np.sum(linridge.coef_ != 0)))

R-squared score (training): 0.931
R-squared score (test): -46.024
Number of non-zero features: 53
R-squared score (training): 0.931
R-squared score (test): 0.818
Number of non-zero features: 53
R-squared score (training): 0.930
R-squared score (test): 0.825
Number of non-zero features: 53
R-squared score (training): 0.927
R-squared score (test): 0.829
Number of non-zero features: 53
R-squared score (training): 0.924
R-squared score (test): 0.829
Number of non-zero features: 53
R-squared score (training): 0.918
R-squared score (test): 0.826
Number of non-zero features: 53
R-squared score (training): 0.911
R-squared score (test): 0.821
Number of non-zero features: 53
R-squared score (training): 0.897
R-squared score (test): 0.807
Number of non-zero features: 53
R-squared score (training): 0.862
R-squared score (test): 0.770
Number of non-zero features: 53
R-squared score (training): 0.807
R-squared score (test): 0.716
Number of non-zero features: 53
R-squared score (training): 0.716
R-sq

## Regression Lasso

In [30]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X_TDM, Y_CONSO,
                                                   random_state = 0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linlasso = Lasso(alpha=2.0, max_iter = 10000).fit(X_train_scaled, y_train)


print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'
     .format(linlasso.score(X_test_scaled, y_test)))

lasso regression linear model intercept: 163.14812369981254
lasso regression linear model coeff:
[ 0.          0.          0.          0.         -0.         -0.
  0.         -0.          0.          0.          0.          0.
  0.          0.          0.         34.50968808  0.          0.
  0.          0.          0.          0.          0.         -0.
  0.          0.          0.82403281  0.         -0.          0.
  7.77131643 -2.02172502 -0.          0.          0.          0.
  0.          0.          0.         40.71843235  0.          0.
  0.         21.45352042  0.          0.          3.07664335  0.
 14.08240315  0.          0.         32.50847927  0.        ]
Non-zero features: 9
R-squared score (training): 0.618
R-squared score (test): 0.519



In [32]:
for alpha in [0.01, 0.1, 0.5,1,2, 3, 5, 10, 20, 50]:
    linlasso = Lasso(alpha, max_iter = 10000).fit(X_train_scaled, y_train)
    r2_train = linlasso.score(X_train_scaled, y_train)
    r2_test = linlasso.score(X_test_scaled, y_test)
    
    print('Alpha = {:.2f}\nFeatures kept: {}, r-squared training: {:.2f}, \
r-squared test: {:.2f}\n'
         .format(alpha, np.sum(linlasso.coef_ != 0), r2_train, r2_test))

Alpha = 0.01
Features kept: 43, r-squared training: 0.93, r-squared test: 0.82

Alpha = 0.10
Features kept: 28, r-squared training: 0.92, r-squared test: 0.82

Alpha = 0.50
Features kept: 19, r-squared training: 0.88, r-squared test: 0.79

Alpha = 1.00
Features kept: 14, r-squared training: 0.79, r-squared test: 0.70

Alpha = 2.00
Features kept: 9, r-squared training: 0.62, r-squared test: 0.52

Alpha = 3.00
Features kept: 5, r-squared training: 0.53, r-squared test: 0.46

Alpha = 5.00
Features kept: 4, r-squared training: 0.42, r-squared test: 0.39

Alpha = 10.00
Features kept: 1, r-squared training: 0.23, r-squared test: 0.21

Alpha = 20.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.00

Alpha = 50.00
Features kept: 0, r-squared training: 0.00, r-squared test: -0.00



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_eveler_conso_jour.sum()

169342.36956521738